In [57]:
import torch
import torch.nn as nn
import os
import numpy as np
import cv2
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
from scipy.ndimage.filters import median_filter

# 언샵 마스크

In [71]:
def unsharp(image, sigma, strength):
 
    # Median filtering
    image_mf = median_filter(image, sigma)
 
    # Calculate the Laplacian
    lap = cv2.Laplacian(image_mf,cv2.CV_64F)
 
    # Calculate the sharpened image
    sharp = image-strength*lap
    
    sharp[sharp>255] = 255
    sharp[sharp<0] = 0
    
    return sharp

In [3]:
# conv function
def conv(c_in, c_out, k_size, stride=2, pad=1, bn=True):
    """Custom convolutional layer for simplicity."""
    layers = []
    layers.append(nn.Conv2d(c_in, c_out, k_size, stride, pad)) # convolution 레이어입니다.
    if bn:
        layers.append(nn.BatchNorm2d(c_out))  # batch normalization 레이어를 추가해줍니다.
    return nn.Sequential(*layers)

# deconv function
def deconv(c_in, c_out, k_size, stride=2, pad=1, bn=True):
    """Custom deconvolutional layer for simplicity."""
    layers = []
    layers.append(nn.ConvTranspose2d(c_in, c_out, k_size, stride, pad))
    if bn:
        layers.append(nn.BatchNorm2d(c_out))
    return nn.Sequential(*layers)

In [59]:
class Generator(nn.Module):
    # initializers
    def __init__(self):
        super(Generator, self).__init__()
        # Unet encoder
        self.conv1 = conv(3, 64, 4, bn=False)
        self.conv2 = conv(64, 128, 4)
        self.conv3 = conv(128, 256, 4)
        self.conv4 = conv(256, 512, 4)
        self.conv5 = conv(512, 512, 4)
        self.conv6 = conv(512, 512, 4)
        self.conv7 = conv(512, 512, 4)
        self.conv8 = conv(512, 512, 4, bn=False)

        # Unet decoder
        self.deconv1 = deconv(512, 512, 4)
        self.deconv2 = deconv(1024, 512, 4)
        self.deconv3 = deconv(1024, 512, 4)
        self.deconv4 = deconv(1024, 512, 4)
        self.deconv5 = deconv(1024, 256, 4)
        self.deconv6 = deconv(512, 128, 4)
        self.deconv7 = deconv(256, 64, 4)
        self.deconv8 = deconv(128, 3, 4)

    # forward method
    def forward(self, input):
        # Unet encoder
        e1 = self.conv1(input)
        e2 = self.conv2(F.leaky_relu(e1, 0.2))
        e3 = self.conv3(F.leaky_relu(e2, 0.2))
        e4 = self.conv4(F.leaky_relu(e3, 0.2))
        e5 = self.conv5(F.leaky_relu(e4, 0.2))
        e6 = self.conv6(F.leaky_relu(e5, 0.2))
        e7 = self.conv7(F.leaky_relu(e6, 0.2))
        e8 = self.conv8(F.leaky_relu(e7, 0.2))
                              
        # Unet decoder
        d1 = F.dropout(self.deconv1(F.relu(e8)), 0.5, training=True)
        d2 = F.dropout(self.deconv2(F.relu(torch.cat([d1, e7], 1))), 0.5, training=True)
        d3 = F.dropout(self.deconv3(F.relu(torch.cat([d2, e6], 1))), 0.5, training=True)
        d4 = self.deconv4(F.relu(torch.cat([d3, e5], 1)))
        d5 = self.deconv5(F.relu(torch.cat([d4, e4], 1)))
        d6 = self.deconv6(F.relu(torch.cat([d5, e3], 1)))
        d7 = self.deconv7(F.relu(torch.cat([d6, e2], 1)))
        d8 = self.deconv8(F.relu(torch.cat([d7, e1], 1)))
        output = torch.tanh(d8)

        return output

In [319]:
def to_var(x):
    """Convert tensor to variable."""
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

def to_data(x):
    """Convert variable to tensor."""
    if torch.cuda.is_available():
        x = x.cpu()
    return x.data

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def show_images(real_a,real_b,fake_b):
    
    plt.figure(figsize=(30,90))
    plt.subplot(131)
    plt.imshow(to_data(real_a).numpy().transpose(1,2,0))
    plt.xticks([])
    plt.yticks([])
    #plt.imsave(f'sketch{num}',(to_data(real_a).numpy().transpose(1,2,0)))
    plt.subplot(132)
    plt.imshow(to_data(real_b).numpy().transpose(1,2,0))
    plt.xticks([])
    plt.yticks([])
    #plt.imsave(f'real_img{num}',(to_data(real_a).numpy().transpose(1,2,0)))
    
    #plt.imsave(f'fake_img{num}',(to_data(real_a).numpy().transpose(1,2,0)))
    plt.subplot(133)
    plt.imshow(to_data(fake_b).numpy().transpose(1,2,0))
    plt.xticks([])
    plt.yticks([])
    #plt.savefig(f'img{num}.jpg',bbox_inches="tight")
    plt.show()
    
def save_images(fake_b,num):
    plt.figure(figsize=(30,90))
    plt.subplot(133)
    plt.imshow(to_data(fake_b).numpy().transpose(1,2,0))
    plt.xticks([])
    plt.yticks([])
    plt.savefig(f'{num}.jpg',bbox_inches="tight")
    plt.show()
    
def upgradepic(a):
    blur = cv2.GaussianBlur(a,(3,3),0)
    median = cv2.medianBlur(blur,5)
    blur2 = cv2.bilateralFilter(median,-1,15,15)
    return blur2;    

# Read Generator

In [391]:
device = torch.device('cpu')
G = Generator()
G.load_state_dict(torch.load("./hed_G_500", map_location=device))

<All keys matched successfully>

In [226]:
import os
os.chdir("D:/facades/skt")

In [409]:
directory_skt = "D:/facades/skt5"

In [385]:
directory_img= "D:/facades/img"

In [386]:
dirlist_img= os.listdir('D:/facades/img')

In [410]:
dirlist_skt = os.listdir('D:/facades/skt5')

In [ ]:
for filename in dirlist_skt:
    facade_a = Image.open(os.path.join(directory_skt,filename)).convert('RGB')
    facade_a = facade_a.resize((256, 256), Image.BICUBIC)
    facade_a = transforms.ToTensor()(facade_a) # Quiz
    facade_a = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(facade_a)
    fake_facade = G(facade_a.expand(1,3,256,256))
    
    facade_b = Image.open(os.path.join(directory_img,filename)).convert('RGB')
    facade_b = facade_b.resize((256, 256), Image.BICUBIC)
    facade_b = transforms.ToTensor()(facade_b) # Quiz
    facade_b = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(facade_b)
    
    
    show_images(denorm(facade_a.squeeze()), denorm(facade_b.squeeze()),denorm(fake_facade.squeeze()),)
    
    save_images(denorm(fake_facade.squeeze()),filename)
    
#     break

# unsharp code

In [87]:
dirlist = os.listdir('C:/Users/User/Desktop/데이터진흥원/d')

In [89]:

for e,i in enumerate(dirlist):
    img2 = os.path.join('C:/Users/User/Desktop/데이터진흥원/d',i)
    original_image = plt.imread(img2)
    sharp1 = np.zeros_like(original_image)
    for i in range(3):
        sharp1[:,:,i] = unsharp(original_image[:,:,i], 5, 0.8)
    
    plt.imsave(f'sharp{e}.jpg',sharp1)
    
    